In [2]:
from Crypto.Cipher import AES
import numpy as np
from Crypto import Random
from Crypto.Random import get_random_bytes
from Crypto.Util import Counter

## General Purpose Functions
These functions are not encryption dependent but are used anywhere in the program 

In [3]:
def xor(str1,str2):
    ''' Gives the XOR of two hex stings with same length'''
    if(len(str1) == len(str2)):
        return "".join(["%x" % (int(x,16) ^ int(y,16)) for (x, y) in zip(str1, str2)])
    else :
        raise ValueError("Enter strings of equal lengths for XOR")
    
def printReadable(string, length):
    ''' Prints the given string by splitting it with spaces for every n letters mentioned in length'''
    try :
        return ' '.join(string[i:i+length] for i in range(0,len(string),length))
    except :
        raise ValueError("Please enter either a string or a hex value of a byte string")

## AES Class 
The following class encloses all the required methods for the program specific to encoding

In [15]:
class aesMeth :
    ''' 
    The following class is a wrapper to the AES class by pyCrypto with AES encryption in mind
    An object of this class can encrypt the data given with different modes of encryption defined in the init 
    '''
    def __init__(self,mode = "ECB",size = 128,ptsize = 80):
        ''' 
        The initialization function :
        Input Method :
        It has the following arguments given to it:
            i)   Mode of operation - ECB by default
            ii)  Key Size - 128, 192 or 256
            iii) Text Size - Has to be a multiple of 16
            
        This function gives the following attributes to an object :
        i)   Key - Randomly generated
        ii)  Plain Text - Randomly generated
        iii) An IV - Randomly generated
        iv)  A Counter - Randomly generated
        v)   The mode of operation :
                Four options available : ECB, CBC, CTR, OFB
        vi)  The Key size - Three options for AES Encryption (128,192,256)
        vii) Text Size - Hes to be a multiple of 16
        '''
        
        if mode != "ECB" and mode != "CBC" and mode != "OFB" and mode != "CTR" :
            raise ValueError("Please enter a valid mode : CBC, ECB, OFB or CTR" )
        else :
            self.mode = mode
            
        if size != 128 and size != 192 and size != 256 :
            raise ValueError("The key should be of either 128 , 192 or 256 bits")
        else :
            self.keySize = size
            
        if ptsize%16 != 0:
            raise ValueError("Please enter value as a multiple of 16 for the text size")
        else:
            self.textSize = ptsize
        
        # This generates an empty instance of the required fields
        self.key = b''
        self.iv = b''
        self.ctr = b''
        self.plaintext = b''
        
        #This calls the functions which gives an initial value to all the fields
        self.genKey()
        self.genPlainText()
        self.genCtr()
        self.genIV()
    
    # Generate functions
    def genKey(self):
        ''' Generates a random key of the given key size'''
        self.key=get_random_bytes(int(self.keySize/8))
        
    def genPlainText(self):
        ''' Generates a random key of the given plain text size'''
        self.plaintext=get_random_bytes(self.textSize)
            
    def genIV(self):
        ''' Generates a random IV'''
        self.iv = get_random_bytes(16)
        
    def genCtr(self):
        ''' Generates a random Control'''
        self.ctr = get_random_bytes(8)
    
    # A single generate function
    '''
    def gen(self,prop):
        if prop == "Key" :
            self.key=get_random_bytes(int(self.keySize/8))
        elif prop == "Plain Text" :
            self.plaintext=get_random_bytes(self.textSize)
        elif prop == "iv" :
            self.iv = get_random_bytes(16)
        elif prop == "Ctr" :
            self.ctr = get_random_bytes(8)            
    '''
    
    # Print functions
    def printKey(self):
        ''' Prints the key in hex form which is readable'''
        print(printReadable(self.key.hex(),8))
        
    def printPt(self):
        ''' Prints the plain text in hex form which is readable'''
        string = self.plaintext.hex()
        length = 8
        for j in range(int(self.textSize/16)):
            print(' '.join(string[i:i+length] for i in range(32*j,32*(j+1),length)))

    def printCtr(self):
        ''' Prints the control in hex form which is readable'''
        print(printReadable(self.ctr.hex(),8))
        
    def printIV(self):
        ''' Prints the IV in hex form which is readable'''
        print(printReadable(self.iv.hex(),8))
        
    #Single print function
    '''
    def gen(self,prop):
        if prop == "Key" :
            print(printReadable(self.key.hex(),8))
        elif prop == "Plain Text" :
            string = self.plaintext.hex()
            length = 8
            for j in range(int(self.textSize/16)):
                print(' '.join(string[i:i+length] for i in range(32*j,32*(j+1),length)))
        elif prop == "Ctr" :
                print(printReadable(self.ctr.hex(),8))
        elif prop == "iv" :
             print(printReadable(self.iv.hex(),8))
    '''
        
    def encrypt(self):
        ''' This function encrypts the given plaintext'''
        if self.mode == "ECB" :
            # ECB is an inbuilt function in pycrypto, hence just applying it
            encObj = AES.new(self.key,AES.MODE_ECB)
            return encObj.encrypt(self.plaintext)
        
        elif self.mode == "CBC" :
            # CBC is an inbuilt function in pycrypto, hence just applying it
            encObj = AES.new(self.key,AES.MODE_CBC,self.iv)
            return encObj.encrypt(self.plaintext)
        
        elif self.mode == "OFB" :
            # OFB is not an inbuilt function, hence applying the algorithm to get the encryption
            # Encrypting each block of data seperately, so using ECB 
            # for each one of them as ECB for one block of code is just AES applied to it
            obfObj = AES.new(self.key,AES.MODE_ECB)
            tempText = obfObj.encrypt(self.iv)
            
            # This stores all the encrypted iv by the equation C_i = f(C_(i-1))
            cArray = np.array([tempText]) 
            for i in range(int(self.textSize/16) - 1) :
                temptext = obfObj.encrypt(cArray[-1])
                cArray = np.append(cArray,temptext)
            
            # This array stores the XOR of the plain text and the encrypted iv
            # The data is converted to its hex form to be able to XOR it
            xorArr = np.array([]) 
            for i in range(int(self.textSize/16)):
                xorArr = np.append(xorArr,xor(cArray[i].hex(),self.plaintext[16*i:16*(i+1)].hex()))
            
            # This string combines all of the encrypted data in a single byte string
            encStr = b'' 
            for i in range(len(xorArr)):
                encStr+=bytes.fromhex(xorArr[i])
            return encStr
        
        elif self.mode == "CTR" :
            # CTR is not an inbuilt function, hence applying the algorithm to get the encryption
            # Encrypting each block of data seperately, so using ECB 
            # for each one of them as ECB for one block of code is just AES applied to it
            ctrObj = AES.new(self.key,AES.MODE_ECB)
            
            # The ctrStr stores the concatenation of the Counter and number
            # Thus effectively being ctrStr[i] = ctr + i
            ctrStr = np.array([])
            for i in range(int(self.textSize/16)):
                appendStr = "00000000"
                # The number in ites entirety must be of length 8, hence mutating it so that it remains of length 8
                try :
                    appendStr = appendStr[:len(appendStr) - len(str(i))]
                except :
                    raise ValueError("Please enter text of smaller size")
                ctrStr = np.append(ctrStr,self.ctr + (appendStr + str(i)).encode('ascii'))
            
            # This string combines all of the encrypted data in a single byte string
            encArr = b''
            for i in range(int(self.textSize/16)):
                encArr += ctrObj.encrypt(ctrStr[i])
            return encArr
        
        else :
            # If none of the given methods are entered
            raise ValueError("Enter a proper method : ECB, CBC, OFB or CTR")

In [16]:
new = aesMeth("ECB",128)
print("Key  :")
new.printKey()
print()
print("Plain Text  :")
new.printPt()
print()
print("IV  :")
new.printIV()
print()
print("Counter  :")
new.printCtr()
print(printReadable(new.encrypt().hex(),32))
#obj = AES.new(new.key,AES.MODE_CBC,new.iv)
#obj.decrypt(ans) == new.plaintext

Key  :
c42abbad 545471f9 b52fc4cc 3b1fcab9

Plain Text  :
36a1a2df 6e589888 da407754 ead9564d
891725ec d6d61674 33b2cbce aafa574a
dc7fc5af 95e4898d 2ed630f2 0ace0db3
bdd7a6c4 1ba98e92 fcf63ff3 80479b98
ba6dc685 97adeeef cfb36061 a53d0702

IV  :
591c0307 c809d4be 845e867c 2e45d424

Counter  :
e70966ef 2a044a54
6efcc18759b4091fc27e1ae901da33f2 d2d87efb529263b2eb2677b7846fd5ec 537a83f2cda1f134acb60bd303bf3d78 95f12b6dbfef7233e6d307a167a86b8e b67bfb6848ece712b92db5062efce9a3


### Validation for the ECB method
The following method is verified when the decryption of the encryted text equals the plain text given

In [22]:
obj1 = aesMeth("ECB",192,96)
aes_obj = AES.new(obj1.key,AES.MODE_ECB)
print("Key  :")
obj1.printKey()
print()
print("Plain Text  :")
obj1.printPt()
print()
print(aes_obj.decrypt(obj1.encrypt()) == obj1.plaintext)

Key  :
139dbe9b 1c75aa35 2db9ca3a a20ab0ec 255378af aa30c86e

Plain Text  :
37ecf16a fe68df62 474a1029 6552441c
3d2021f5 d4c5a5a4 348ff1dc 33cfc11f
c789e7e6 f1f75c5c c1e8a318 955fc2dd
3e675017 f1bab63e eb1de207 ebd1f051
901d1b32 22a4a523 f02ce1c6 75982b00
b2bc9d37 13c204df acb95db8 78e303fc

True


### Validation for the CBC method
The following method is verified when the decryption of the encryted text equals the plain text given

In [23]:
obj1 = aesMeth("CBC",192,96)
aes_obj = AES.new(obj1.key,AES.MODE_CBC,obj1.iv)
print("Key  :")
obj1.printKey()
print()
print("Plain Text  :")
obj1.printPt()
print()
print("IV  :")
obj1.printIV()
print()
print(aes_obj.decrypt(obj1.encrypt()) == obj1.plaintext)

Key  :
4a40df8c 8a921786 c7f1df1f b2578cb2 ae99aaf1 36ce2307

Plain Text  :
49ea98dd 17e53e3f 1e9f3fcb 9f813b7e
b57fa92d 9b916070 4e6ae16a 0c5370fe
9da12a95 0a49ce3f b5055fcf 6d239c59
96af31d0 ed7800f1 4deb1c98 9c7aa4fe
7a1f8346 3523b244 3b8a99e1 596744de
0c250aa3 0f7177a2 1e6e2f16 5188ea29

IV  :
930952e1 57138ebf f05644ba 2c000747

True


### OFB Method 

In [19]:
new = aesMeth("OFB",128,96)
print("Key  :")
new.printKey()
print()
print("Plain Text  :")
new.printPt()
print()
print("IV  :")
new.printIV()
print()
ans = new.encrypt().hex()
print("Encrypted Data  :")
for i in range(int(len(ans)/32)):
    print(printReadable(ans[32*i:32*(i+1)],8))

Key  :
1fe4a060 336571ff f6581fc8 4e47190d

Plain Text  :
374a3811 0b948a0e be4a3305 7bc7b57a
31c494ae 355c3923 665d227b 1d2c3dc4
f26119bc 60de5487 dfb10551 c9ecd84b
497a637a d00883e3 2383b9a9 92f0e218
72a7acdd 68ae4fd0 88051598 59d85793
92114d97 c7691017 865a5795 6f81b085

IV  :
2da0a274 25786192 f43d4213 dc2e67e0

Encrypted Data  :
49969aaf 9e7fd432 1592bfe6 8f025e3a
ac1147d9 a57a2506 46c7815b 111a88ff
bf30896f 48656a6d 8007adbc be53f8ec
ba692aba 18cf448f 3cc7baa8 746aa193
a3188777 c8a261c5 e0011984 de93ca94
0684e2f5 0d68b7ca 8fff87c9 afd98a55


## Ctr Method

In [20]:
new = aesMeth("CTR",128,96)
print("Key  :")
new.printKey()
print()
print("Plain Text  :")
new.printPt()
print()
print("Counter  :")
new.printCtr()
ans = new.encrypt().hex()
print()
print("Encrypted Data  :")
for i in range(int(len(ans)/32)):
    print(printReadable(ans[32*i:32*(i+1)],8))

Key  :
3fb7933f 7bdf0445 097312d7 57e081dd

Plain Text  :
4a379380 2f02fd99 03001807 83f77990
9e256529 d776d1e8 78b43875 8204265b
2b5bfad1 52ec17e9 4ebb2ae7 caf12f2d
a5543677 7ac60340 b3f75a20 a8db23fe
381fcc3a dd786745 a003bd52 412f4ee3
779d68a6 b45727c8 01b2899f 05b5723e

Counter  :
18459974 b8ca57fe

Encrypted Data  :
81e5ca8d 40de5e25 02c94692 6106816f
8cad6b82 25a50127 1d999fc5 34e47478
2dd29af4 6cbb7493 07058d79 b5de83fb
f011b9f1 75b0385b 1932b484 1b98e264
85791ae7 3224ee6d c2fad230 40a7b769
ede3fe8a ea3e60c9 e903498b edce14bc


## Playground for Experimentation
Rough work - Not required for the working of the main program

In [586]:
?aesMeth

In [587]:
size = 192
mode = "ECB"
mainStr =b''
print("The Plain Text  :")
for i in range(5):
    for j in range(4):
        plaintext = get_random_bytes(4)
        print(plaintext.hex(), end = " ")
        mainStr+=plaintext
    print("")
    
print()
print("The Key  :")
keySize = 192
key = b''
for i in range(int(keySize/32)):
    keyTemp=get_random_bytes(4)
    print(keyTemp.hex(), end = " ")
    key+=keyTemp
    
print()
print()
ivStr=b''
print("The IV")
for j in range(4):
    ivTemp = get_random_bytes(4)
    print(ivTemp.hex(), end = " ")
    ivStr+=ivTemp
print("")

print()
ctrStr=b''
print("The CTR  :")
for j in range(2):
    ctrTemp = get_random_bytes(4)
    print(ctrTemp.hex(), end = " ")
    ctrStr+=ctrTemp
print("")

The Plain Text  :
c79f8158 b19b23da 03f2474a 6fccd243 
e7187886 55e773f9 dc5c0e1b 3d241312 
522da52b e94fcb58 449d4ff0 be4fdbfc 
a0b8919b 28e41e91 555b64fb 74adf3ad 
f3aaeafe 23fa19f9 fac83dab 4828b061 

The Key  :
9f31760f fc7e79c5 fcda5b31 f2fd5015 7db5cbd3 2b040b9d 

The IV
673645c8 72315cd2 2fc1a4f9 3b78ff03 

The CTR  :
3841a760 08136157 


In [588]:
def hexxor(a, b):
    return "".join(["%x" % (int(x,16) ^ int(y,16)) for (x, y) in zip(a, b)])

def encrypt(string, length):
    return ' '.join(string[i:i+length] for i in range(0,len(string),length))

In [589]:
print("Give input for XOR")
xorStr1 = key.hex()
xorStr2 = ivStr.hex()
xor = hexxor(xorStr1,xorStr2)
for i in range(len(xor)):
    print(xor[i],end = "")
    if i % 8 == 0 and i != 0:
        print(" ",end = "")
print()

Give input for XOR
f80733c78 e4f2517d 31bffc8c 985af16


In [590]:
hex1 = '6fa75d49722276a11a8748aca562755a'
hex2 = '75a805595b2da0a8f56b7f1f471bcf3a'
x = hexxor(hex1,hex2)
print(encrypt(x,8))

1a0f5810 290fd609 efec37b3 e279ba60


In [591]:
obj = AES.new(key, AES.MODE_ECB)

In [592]:
encrypt(obj.encrypt(mainStr).hex(),8)

'aa1fab64 3a32e4f2 728e007c fbaee6e3 aeed900a de4a3b4f cd302953 9a4b7fe6 2a01aba4 056f0abd 6029ad55 f44cd413 6645331e e237d0cc 4dfe61f9 0a1d7ba9 c8832844 93d4371a ef21c56f 757d7ba0'

In [593]:
obj.decrypt(obj.encrypt(mainStr))

b'\xc7\x9f\x81X\xb1\x9b#\xda\x03\xf2GJo\xcc\xd2C\xe7\x18x\x86U\xe7s\xf9\xdc\\\x0e\x1b=$\x13\x12R-\xa5+\xe9O\xcbXD\x9dO\xf0\xbeO\xdb\xfc\xa0\xb8\x91\x9b(\xe4\x1e\x91U[d\xfbt\xad\xf3\xad\xf3\xaa\xea\xfe#\xfa\x19\xf9\xfa\xc8=\xabH(\xb0a'

In [594]:
mainStr

b'\xc7\x9f\x81X\xb1\x9b#\xda\x03\xf2GJo\xcc\xd2C\xe7\x18x\x86U\xe7s\xf9\xdc\\\x0e\x1b=$\x13\x12R-\xa5+\xe9O\xcbXD\x9dO\xf0\xbeO\xdb\xfc\xa0\xb8\x91\x9b(\xe4\x1e\x91U[d\xfbt\xad\xf3\xad\xf3\xaa\xea\xfe#\xfa\x19\xf9\xfa\xc8=\xabH(\xb0a'

In [595]:
aes = AES.new(key, AES.MODE_CBC, ivStr)
data = 'hello world 1234hello world 1234hello world 1234hello world 1234' # <- 16 bytes
encd = aes.encrypt(mainStr)
aes = AES.new(key, AES.MODE_CBC, ivStr)
decd = aes.decrypt(encd)
print (decd)
decd == mainStr

b'\xc7\x9f\x81X\xb1\x9b#\xda\x03\xf2GJo\xcc\xd2C\xe7\x18x\x86U\xe7s\xf9\xdc\\\x0e\x1b=$\x13\x12R-\xa5+\xe9O\xcbXD\x9dO\xf0\xbeO\xdb\xfc\xa0\xb8\x91\x9b(\xe4\x1e\x91U[d\xfbt\xad\xf3\xad\xf3\xaa\xea\xfe#\xfa\x19\xf9\xfa\xc8=\xabH(\xb0a'


True

In [596]:
count = Counter.new(128,initial_value = int(ctrStr.hex(),16))
ctr_obj = AES.new(key,AES.MODE_CTR,counter = count)

In [597]:
prop = ctr_obj.encrypt("abcd")
ans = ctr_obj.decrypt(prop)
ans = ctr_obj.decrypt(prop)
ans = ctr_obj.decrypt(prop)
ans = ctr_obj.decrypt(prop)

In [598]:
obj = AES.new(key, AES.MODE_ECB)
temp = encrypt(obj.encrypt(ctrStr))
hexxor(temp, mainStr)

ValueError: Input strings must be a multiple of 16 in length

In [599]:
pt = bytes.fromhex(hex(int(ctrStr.hex(),16) + 1))
print(hex(int(ctrStr.hex(),16) + 1))

ValueError: non-hexadecimal number found in fromhex() arg at position 1

In [600]:
obj.encrypt(pt)

b'\xf6|\x1a\xeb\xc7s\xd2\xa4KH\x8e)J@\xd0\x8a'

In [601]:
int(count().hex(),16)

4053705170504343897

In [602]:
pt = (str(ctrStr.hex()) + "00000001").encode()
pt

b'3841a7600813615700000001'

In [603]:
hexxor(obj.encrypt(pt).hex(),mainStr[:8].hex())

ValueError: Input strings must be a multiple of 16 in length

In [604]:
obfObj = AES.new(key,AES.MODE_ECB)
temptext = obfObj.encrypt(ivStr)
x = np.array([temptext])
for i in range(int(80/16) - 1) :
    temptext = obfObj.encrypt(x[-1])
    x = np.append(x,temptext)

In [605]:
y = np.array([])
for i in range(int(80/16)):
    y = np.append(y,hexxor(x[i].hex(),mainStr[i:i+8].hex()))

In [606]:
y

array(['e26484fe630ccb4f', '5c0190bc23c10270', '51d015dda0c70899',
       'f812a4526087698e', 'ff96354c44e7cf1d'], dtype='<U32')

In [607]:
#    def gen(self,prop,*args):
#        if prop == "Key" :
#            self.key=get_random_bytes(self.keySize/8)
#        elif prop == "Plain Text" :
#            if len(args) == 0:
#                raise "For Plain Text the text size is required to be entered"
#            else :
#                if args[0]%16 != 0:
#                    raise "Please enter value as a multiple of 16 for the text size"
#                else:
#                    self.plaintext=get_random_bytes(args[0])
#        elif prop == "IV":
#            self.iv = get_random_bytes(16)
#        elif prop == "CTR":
#            self.ctr = get_random_bytes(16)

In [608]:
# Finding the CTR method
new = ctrStr + b'00000001'
new

b'8A\xa7`\x08\x13aW00000001'

In [609]:
ansStr = np.array([])
for i in range(int(80/16)):
    ansStr = np.append(ansStr,ctrStr + ("0000000" + str(i)).encode('ascii'))
ansStr

array([b'8A\xa7`\x08\x13aW00000000', b'8A\xa7`\x08\x13aW00000001',
       b'8A\xa7`\x08\x13aW00000002', b'8A\xa7`\x08\x13aW00000003',
       b'8A\xa7`\x08\x13aW00000004'], dtype='|S32')

In [610]:
finNewObj = AES.new(key,AES.MODE_ECB)
finStr = np.array([])
for i in range(len(ansStr)):
    finStr = np.append(finStr,finNewObj.encrypt(ansStr[i]))

In [611]:
finStr

array([b'\x19Z\xe3\xd6\x1f\x05\x03\xb5Q\xac\x15\xe2\x1fip\xee',
       b'h\x93\xa7i\xea\xe8:\xe2k\xe9\xbe\x1e\xf8\xc1\xd4a',
       b'%\xc1\xd1\xe3\xca\xaf\xfb\x8c\xb0Py\xc3\xb2\x8e^\xf8',
       b'y\xf8S\x19b\xff\xf8a>\xef\xd2\xee\xf7\xba\xf5\xb5',
       b'\x8c\x08\xc6\xab\xe3\x1e\xc3@\x1bdIR\xef\xeay\\'], dtype='|S32')

In [612]:
i = 100
a = "00000000"
len(str(i))
a = a[:len(a) - len(str(i))]